<a href="https://colab.research.google.com/github/SuperMusey/FoundationOfPrivacy/blob/main/MIA_phase2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MIA FOR Finetuned LLM

In [1]:
# update the downloading command as my LFS runs out so cannot directly clone model.safetensors
%cd /content

!git clone https://github.com/2020pyfcrawl/18734-17731_Project_Phase2_3.git

%cd /content/18734-17731_Project_Phase2_3


/content
Cloning into '18734-17731_Project_Phase2_3'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 43 (delta 2), reused 5 (delta 2), pack-reused 36 (from 1)
Receiving objects: 100% (43/43), 55.69 MiB | 38.77 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/18734-17731_Project_Phase2_3


### Variables and libraries

In [ ]:
# install the required libraries if you have not done so (on you local machine or GPU server)
# you may not need to run this if you use colab as they are pre-installed, but you can always do it.
%pip install -r requirements.txt

In [2]:
import os, math, argparse
os.environ.setdefault("TRANSFORMERS_NO_TORCHVISION", "1")

import torch
import numpy as np
from datasets import load_from_disk
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import roc_auc_score, roc_curve, auc as _auc
import matplotlib.pyplot as plt
import json
from pathlib import Path
from datasets import Dataset

In [3]:
# global variable, check the current position to adjust the path
phase = "train" # or train / val / final
target_model_dir = f"./models/{phase}/gpt2_3_lora32_adamw_b8_lr2"
data_dir = f"./data/{phase}/"
batch_size = 50

# you may change block size if you like (max length for the tokenizer below)
block_size = 512

### Data pre-processing

In [4]:
def tokenize_dataset(ds, tok, max_len):
    ds = ds.filter(lambda ex: ex.get("text", None) and len(ex["text"].strip()) > 0)

    def _map(ex):
        out = tok(ex["text"], truncation=True, padding=True, max_length=max_len, return_attention_mask=True)
        out["labels"] = out["input_ids"].copy()
        return out

    ds = ds.map(_map, batched=True, remove_columns=ds.column_names)
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return ds

def _read_json(path: Path):
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)

In [5]:
# for tests, you may only load a part of the data to save time while implementing,
# as running all 2000 samples on CPU may be slow, but not a problem here for GPU

# load test data
data_dir = Path(data_dir)
test_path = data_dir / "test.json"
test_items = _read_json(test_path)
ds_test = Dataset.from_dict({"text": test_items})

# tokenizer the test data
tokenizer = AutoTokenizer.from_pretrained(target_model_dir, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side="right"

ds_test = tokenize_dataset(ds_test, tokenizer, block_size)
dl_test = DataLoader(ds_test, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# you may load the model using the code:

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
model = AutoModelForCausalLM.from_pretrained(target_model_dir, dtype="auto").to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### MIA

Implement your attack here! \
Hint: use shadow models.

In [ ]:
# implement your attack here
@torch.no_grad()
def your_attack(

):
    pass

scores_test = your_attack(...)

### Result

In [ ]:
# load the label here to compute the performance, you will only have full access to the label in train set to test your method
if phase == "train":
    label_path = data_dir / "test_label.json"
    label_items = _read_json(label_path)

    y_true = np.array(label_items)
    scores = np.array(scores_test)
    fpr, tpr, thr = roc_curve(y_true, scores)
    auc_val = roc_auc_score(y_true, scores)
    print(auc_val)

    print(max(tpr[fpr < 0.01])) # TPR @ 0.01FPR
    # WE ONLY CARE TPR @ 0.01FPR!!! SO INCREASE THIS AS MUCH AS POSSIBLE!!!
elif phase == "val" or phase == "final":
    pred_path = data_dir / "prediction.csv"
    with open(pred_path, "w", encoding="utf-8") as f:
        for s in scores_test:
            f.write(json.dumps(float(s), ensure_ascii=False) + "\n")
else:
    print("Wrong phase.")

### Packaging the submission

zip the prediction file and upload to the leaderboard.

In [ ]:
import os
from pathlib import Path
import zipfile

with zipfile.ZipFile(f"project_submission.zip", 'w') as zipf:
    for phase in ["val", "final"]:
        data_dir = f"./data/{phase}/"
        data_dir = Path(data_dir)

        file = data_dir / "prediction.csv"
        if file.exists():
            arcname = os.path.join(phase, file.name)
            zipf.write(file, arcname=arcname)
        else:
            raise FileNotFoundError(f"`prediction.csv` not found in {data_dir}.")

### Visualization

A few visualizations that may help you develop your method and write reports.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

eps = 1e-12
fpr_ = np.clip(fpr, 1e-5, 1)
tpr_ = np.clip(tpr, 1e-5, 1)

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(fpr_, tpr_, lw=2, label=f'ROC (AUC = {auc_val:.4f})')
ax.plot([1e-5, 1], [1e-5, 1], lw=2, ls='--', label='Chance')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1e-5, 1.0)
ax.set_ylim(1e-5, 1.0)

ticks = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.get_xaxis().set_minor_formatter(plt.NullFormatter())
ax.get_yaxis().set_minor_formatter(plt.NullFormatter())

ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('MIA ROC (log–log focus on small FPR/TPR)')
ax.legend(loc='lower right')
ax.grid(True, which='both', alpha=0.5)

plt.show()

In [ ]:
# draw distribution
import matplotlib.pyplot as plt
import seaborn as sns

# y_true = np.array(label_items)
# scores = np.array(scores_test)

scores_mem = scores[y_true == 1]
scores_non = scores[y_true == 0]

plt.figure(figsize=(12, 6))
sns.histplot(scores_mem, bins=50, color='salmon', kde=True, label='member')
sns.histplot(scores_non, bins=50, color='skyblue', kde=True, label='non-member')

threshold_value = np.percentile(scores_non, q=99)
print(threshold_value)
plt.axvline(
    x=threshold_value,
    color='purple',
    linestyle='--',
    linewidth=2,
    label=f'0.01 FPR: {threshold_value:.2f}'
)


plt.title('Loss distribution', fontsize=16)
plt.xlabel('Loss', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.legend(fontsize=10)

plt.show()

In [ ]:
# draw ROC curve and attach the figure in the report
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_val:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance line')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title(f'MIA ROC Curve for Train Data')
plt.legend(loc="lower right")
plt.grid(alpha=0.5)
plt.show()

**For Shadow Data**

In [7]:
#!/usr/bin/env python
# prepare_data.py
# curate training data for the model

import os, json, random
from pathlib import Path
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer

OUTDIR = "shadow_data/shadow_0"
SEED = 42
TRAIN_PER_SRC = 10_000
MIN_TOKENS = 25

def set_seed_all(seed: int):
    import numpy as np
    random.seed(seed); np.random.seed(seed)
    try:
        import torch
        torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    except Exception:
        pass

def ensure_text_column(ds: Dataset, src: str) -> Dataset:
    if src == "wikitext103":
        assert "text" in ds.column_names
        return ds.remove_columns([c for c in ds.column_names if c != "text"])
    raise ValueError(src)

def basic_clean(ds: Dataset) -> Dataset:
    ds = ds.filter(lambda ex: isinstance(ex.get("text", None), str) and len(ex["text"].strip()) > 0)
    def _strip_map(ex): return {"text": " ".join(ex["text"].split())}
    return ds.map(_strip_map, batched=False)

def filter_by_tokens(ds: Dataset, tok, min_tokens: int) -> Dataset:
    def _len_map(batch):
        enc = tok(batch["text"], add_special_tokens=False)
        return {"_tok_len": [len(ids) for ids in enc["input_ids"]]}
    ds = ds.map(_len_map, batched=True)
    ds = ds.filter(lambda ex: ex["_tok_len"] >= min_tokens)
    return ds.remove_columns(["_tok_len"])

def sample_n(ds: Dataset, n: int, seed: int):
    n = min(n, len(ds))
    idx = list(range(len(ds)))
    random.Random(seed).shuffle(idx)
    take = sorted(idx[:n])
    return ds.select(take), set(take)

def dump_json(path: Path, obj):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

def main():
    set_seed_all(SEED)
    os.makedirs(OUTDIR, exist_ok=True)

    tok = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token

    # ---------- Load & filter (WikiText-103-raw-v1) ----------
    wiki_raw = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1")["train"]
    wiki = ensure_text_column(wiki_raw, "wikitext103")
    wiki = basic_clean(wiki)
    wiki = filter_by_tokens(wiki, tok, MIN_TOKENS)

    # train set
    wiki_train, wiki_train_idx = sample_n(wiki, TRAIN_PER_SRC, SEED + 1)

    out_dir = Path(OUTDIR)
    train_json = [{"text": ex["text"]} for ex in wiki_train]
    dump_json(out_dir / "train_finetune.json", train_json)

    print("[OK] JSON saved to", OUTDIR)

if __name__ == "__main__":
    main()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Map:   0%|          | 0/1165029 [00:00<?, ? examples/s]

Map:   0%|          | 0/1165029 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1063 > 1024). Running this sequence through the model will result in indexing errors


Filter:   0%|          | 0/1165029 [00:00<?, ? examples/s]

[OK] JSON saved to shadow_data/shadow_0


In [ ]:
# Save to cloud
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Copy shadow data to Drive
!cp -r shadow_data/ /content/drive/MyDrive/FoundationOfPrivacy/shadow_data/

In [ ]:
# To load back:
!cp -r /content/drive/MyDrive/FoundationOfPrivacy/shadow_data/ ./